# Rotations Only Webapp Devo:

A notebook for developing a webapp that only does rotations in 3D space, no boosts. Here is a mockup.

![](Design/rotation_only_mockup.png)

The goal is to compare the output of the Rodrigues rotation function to that for what I have claimed is a generalization of the Rodrigues function. Why should this be so? The reason is simple. Let the norm of _h_ be unity and the first term of _h_ be zero. In that case:

$$ U q U^* + \frac{1}{2}( (U U q)^* - (U^* U^* q)^*) = U q U^* $$

For this particular choice of _h_, the Rodrigues function and the generalization of the Rodrigues function should do the same transformation. Let's see if we can create images of what is going on.

In [ ]:
%%capture
%matplotlib inline
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import math
import unittest
from types import FunctionType
from typing import Union

import pandas as pd

# To get equations the look like, well, equations, use the following.
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from IPython.display import display

# Tools for manipulating quaternions.
from Qs import Q, Qs, abs_of_vector, sin, sins, add, adds, norm_squared, qrandom, rotation_only, squares, norm_squareds

from IPython.core.display import display, HTML, Math, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

The output of the function rotation() and rotation_and_or_boost() must be applied again and again and again. Write a function to generate a quaternion series giving the name of a function and a quaternion and/or another function.

In [ ]:
def generate_Qs(func: FunctionType, q_1: Union[Q, FunctionType], dim: int = 10, qs_type: str = "ket"):
    """
    One quaternion cannot tell a story. generate_Qs provides a general way to create a
    quaternion series given a function and one quaternion/another function. The function
    is applied to each subsequent value of the function. If q_1 is itself a function, it
    will be called each time.
    
    Args:
        func: FunctionType   a function that generates an instance of the class Q
        q_1: Q, FunctionType  Either an instance of Q or a Q function
        dim: int    The dimensions of the quaternion series
        qs_type:    bra/ket/operator  Only works for a square operator at this time
        
    Returns: Qs
    
    """
    
    if type(q_1) == Q:
        new_qs = [func(q_1)]
    
        for _ in range(dim-1):
            new_qs.append(func(new_qs[-1]))
            
    elif type(q_1) == FunctionType:
        new_qs = [func(q_1())]
    
        for _ in range(dim-1):
            new_qs.append(func(q_1()))
                       
    else:
        raise ValueError(f"Cannot work with q_1's type: {type(q_1)}")
        
    return Qs(new_qs, qs_type=qs_type)

In [ ]:
test_q = generate_Qs(sin, Q([.1, .2, .3, .4]))
test_q.print_state("test q")

Show this result is generated by apply sin() to each step.

In [ ]:
sin(Q([.1, .2, .3, .4])).print_state("sin")
sin(sin(Q([.1, .2, .3, .4]))).print_state("sin sin")
sin(sin(sin(Q([.1, .2, .3, .4])))).print_state("sin sin sin")

Test passing in a function for q_1:

In [ ]:
test_random = generate_Qs(sin, qrandom)
test_random.print_state("test random")

In [ ]:
def generate_QQs(func, q_1, q_2, dim=10, qs_type="ket"):
    """
    One quaternion cannot tell a story. generate_QQs provides a general way to create a
    quaternion series given a function and two other quaternions/functions. The function
    is applied to each subsequent value of the function. If q_1 or q_2 is itself a function, it
    will be called each time.
    
    This function was written for the function add to be
    able to represent inertial motion, adding the same value over and over again.
    
    Args:
        func: FunctionType   a function that generates an instance of the class Q
        q_1: Q, FunctionType  Either an instance of Q or a Q function
        q_2: Q, FunctionType  Either an instance of Q or a Q function
        dim: int    The dimensions of the quaternion series
        qs_type:    bra/ket/operator  Only works for a square operator at this time
        
    Returns: Qs
    
    """
        
    if (type(q_1) == Q) and (type(q_2) == Q):
        
        new_qs = [func(q_1, q_2)]
    
        for _ in range(dim-1):
            new_qs.append(func(new_qs[-1], q_2))
            
    elif ((type(q_1) == Q) and (type(q_2) == FunctionType)):
        new_qs = [func(q_1, q_2())]
    
        for _ in range(dim-1):
            new_qs.append(func(new_qs[-1], q_2()))
    
    elif ((type(q_1) == FunctionType) and (type(q_2) == Q)):
        new_qs = [func(q_1(), q_2)]
    
        for _ in range(dim-1):
            new_qs.append(func(q_1(), new_qs[-1]))
                          
    elif ((type(q_1) == FunctionType) and (type(q_2) == FunctionType)):
        new_qs = [func(q_1(), q_2())]
    
        for _ in range(dim-1):
            new_qs.append(func(q_1(), q_2()))
                       
    else:
        raise ValueError(f"Cannot work with q_1's type: {type(q_1)}")
        
    return Qs(new_qs, qs_type=qs_type)
    
    new_qs = [func(q_1, q_2)]
    
    for _ in range(dim-1):
        new_qs.append(func(new_qs[-1], q_2))
        
    return Qs(new_qs, qs_type=qs_type)

In [ ]:
a10 = generate_QQs(add, Q([10, 9, 8, 7]), Q([0.1, -0.2, 0, 0]))
a10.print_state("a10")

In [ ]:
a10r = generate_QQs(add, Q([10, 9, 8, 7]), qrandom)
a10r.print_state("a10 random")

In [ ]:
a10r = generate_QQs(add, qrandom, Q([10, 9, 8, 7]))
a10r.print_state("a10 random")

In [ ]:
arr = generate_QQs(add, qrandom, qrandom)
arr.print_state("a10 random")

In [ ]:
df = pd.DataFrame(generate_QQs(add, qrandom, qrandom).xyz())

In [ ]:
df

In [ ]:
sq_1123 = generate_QQs(rotation_only, Q([1, 1, 2, 3]), qrandom)
rot_squares = squares(sq_1123)
rot_norm_squared = norm_squareds(sq_1123)

In [ ]:
rot_squares_df = pd.DataFrame(rot_squares.df)
rot_norm_squared_df = pd.DataFrame(rot_norm_squared.df)

In [ ]:
rot_squares.df

In [ ]:
rot_norm_squared.df

In [ ]:
q10987 = Q([10, 9, 8, 7])
q1234 = Q([1, 2, 3, 4])

In [ ]:
newdf = pd.DataFrame([[1234, q10987])

In [ ]:
newdf

In [ ]:
newdf = pd.DataFrame([q1234.df, q10987.df])
newdf

In [ ]:
newdf = pd.DataFrame(q1234, q10987.df.to_numpy())
newdf

In [ ]:
newdf = pd.DataFrame([[q1234], [q10987]])
newdf

In [ ]:
newdf = pd.DataFrame([[q1234.t, q1234.x, q1234.y, q1234.z], [q10987.t, q10987.x, q10987.y, q10987.z]])
newdf

In [ ]:
q1234.df

In [ ]:
q2 = [q1234, q10987]

In [ ]:
d2 = [[q.t, q.x, q.y, q.z] for q in q2]

In [ ]:
d2

In [ ]:
d2_df = pd.DataFrame(d2)

In [ ]:
d2_df

In [ ]:
for q in sq_1123.qs:
    norm_squared(q).print_state("rotated ||q||^2")